# Plant Disease Detection Training
This notebook trains an EfficientNetV2-S model on the PlantVillage dataset using PyTorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

## Dataset Discovery
Automatically locate the dataset and train/validation directories.

In [ ]:
# find dataset
base_input = "/kaggle/input"
dataset_path = None
for folder in os.listdir(base_input):
    if any(x in folder.lower() for x in ["plant", "disease", "village"]):
        dataset_path = os.path.join(base_input, folder)
        break

if dataset_path is None:
    raise Exception("Dataset not found")

print("Dataset:", dataset_path)

# find train / val dirs
train_dir, val_dir = None, None
for root, dirs, _ in os.walk(dataset_path):
    low = [d.lower() for d in dirs]
    if "train" in low and ("valid" in low or "val" in low):
        train_dir = os.path.join(root, dirs[low.index("train")])
        val_dir = os.path.join(root, dirs[low.index("valid")] if "valid" in low else os.path.join(root, dirs[low.index("val")]))
        break

if train_dir is None or val_dir is None:
    raise Exception("Train/Val folders missing")

print("Train:", train_dir)
print("Val:", val_dir)

## Data Loading
Set up transforms and data loaders.

In [ ]:
# data
IMAGE_SIZE = 160
BATCH = 32
EPOCHS = 15

transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

class_names = train_ds.classes
print("Classes:", len(class_names))

## Model Setup
Initialize EfficientNetV2-S and modify the classifier.

In [ ]:
# model
model = models.efficientnet_v2_s(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scaler = torch.amp.GradScaler(device="cuda")

## Training Loop
Train the model with mixed precision.

In [ ]:
# train
train_acc_list, val_acc_list = [], []

for epoch in range(EPOCHS):
    model.train()
    correct, total = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda"):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    train_acc_list.append(train_acc)

    # validate
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_acc = 100 * correct / total
    val_acc_list.append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train: {train_acc:.2f}% | Val: {val_acc:.2f}%")

## Save & Plot
Save the trained model and plot accuracy.

In [ ]:
# save
torch.save(model.state_dict(), "plant_disease_model.pth")
print("Saved: plant_disease_model.pth")

# plot
plt.plot(train_acc_list, label="Train Acc")
plt.plot(val_acc_list, label="Val Acc")
plt.legend()
plt.show()